Необходимо на заданном наборе данных провести его преобразование к формату, пригодном для построения модели линейной регрессии. Затем по каждой уникальной комбинации значений переменных Object и Energy_Source в рамках каждого уникального значения переменной Organization произвести расчет следующих параметров модели линейной регрессии:
<ul>
<li>Значение свободного члена модели (записать только для переменной с названием Consumption, для остальных заполнить nan)</li>
<li>Коэффициенты регрессии для переменных модели (кроме свободного члена)</li>
<li>Значения p-value для t-теста о значимости коэффициентов регрессии (кроме свободного члена)</li>
<li>Бинарный индикатор прохождения коэффициентами регрессии t-теста о значимости (кроме свободного члена)</li>
<li>Значение p-value для F-теста о значимости регрессии в целом (записать только для переменной с названием Consumption, для остальных заполнить nan)</li>
<li>Значение скорректированного коэффициента детерминации R2 Adjusted (записать только для переменной с названием Consumption, для остальных заполнить nan)</li>
</ul>

Результаты всех моделей записать в единый датафрейм pandas. 

В расчетах не учитывать строки со значением переменной Consumption=0.<p>
В случае отсутствия данных для расчета модели выдавать датафрейм с nan значениями.  

In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools
import warnings
warnings.filterwarnings("ignore")

### Без создания формулы модели ###

def make_new_DF(initialDF):
### Создает датафрейм с переменными - столбцами ###
    categorials = initialDF.columns.tolist()[:2]	# берем объект и энергоресурс
    features = initialDF.Feature.unique().tolist()	# берем названия переменных
    new_df_cols = categorials+features	# соединяем
    new_DF = pd.DataFrame(data=None, columns=new_df_cols)	# создаем датафрейм с новыми столбцами 
    return new_DF

def fill_new_DF(initialDF, new_DF, cat1, cat2):
### Заполняет созданный датафрейм данными из исходного ###
    features = initialDF.Feature.unique().tolist()	# список переменных
    for feature in features:
        new_DF[feature]=initialDF.Value[initialDF.Feature==feature].values	# заполняем столбец каждой переменной данными
    new_DF.Object.fillna(value=cat1, inplace=True)	# заполняем значение объекта
    new_DF.Energy_Source.fillna(value=cat2, inplace=True)	# заполняем значение энергоресурса
    return new_DF

def generate_out_df(model, Cube_Variables, object, output_DF_cols):
### Генерирует выходной датафрейм на основании расчитанной модели ###
### Заполняет значения пустого пересечения показателей*переменных null-значением ###
    output_DF = pd.DataFrame(data=None, columns=output_DF_cols)
    null_indicator = np.nan

    # заполняем значения по переменным из модели
    output_DF['Cube_Variable'] = Cube_Variables[1:]
    output_DF["coef"] = model.params[1:].tolist()
    output_DF["p_value"] = model.pvalues[1:].tolist()
    output_DF["Usefull"] = np.where(output_DF["p_value"]<=threshold, 1, 0)
    output_DF["Object"] = object
    output_DF["p_value"].loc[output_DF.p_value.isnull()==True]=1
    output_DF['r2_adj'] = null_indicator
    output_DF['Intercept'] = null_indicator
    output_DF['Significance'] = null_indicator
    
    # заполняем общие значения по модели 
    general_values = pd.DataFrame([[object, Cube_Variables[0], model.rsquared_adj, model.params[0], model.f_pvalue, null_indicator, null_indicator, null_indicator]], columns=output_DF_cols)
    output_DF = output_DF.append(general_values, ignore_index=True)
    return output_DF

def generate_blank_out_df(Cube_Variables, object, output_DF_cols):
### Генерирует пустой выходной датафрейм, если расчет ведется на пустых данных ###
### Заполняет значения коэффициентов нулями, все остальные числовые показатели заполняет null-значением ###
    output_DF = pd.DataFrame(data=None, columns=output_DF_cols)
    null_indicator = np.nan

    output_DF['Cube_Variable'] = Cube_Variables
    output_DF["coef"] = 0
    output_DF["p_value"] = null_indicator
    output_DF["r2_adj"] = null_indicator
    output_DF['Intercept'] = null_indicator
    output_DF["Usefull"] = null_indicator
    output_DF["Object"] = object
    output_DF["Significance"] = null_indicator
    return output_DF

/Users/maximhorin/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_excel("sample_data.xlsx")

In [3]:
df

,Object,Energy_Source,Feature,Value,Date,Organization,Cube_Variable,p_value_threshold
0,Obj1,Energy source 1,Consumption,730691,01.01.17,Org1,Energy source 1 - Consumption,0.1
1,Obj1,Energy source 1,Variable_1,953,01.01.17,Org1,Energy source 1 - Variable 1,0.1
2,Obj1,Energy source 1,Variable_2,499,01.01.17,Org1,Energy source 1 - Variable 2,0.1
3,Obj1,Energy source 1,Variable_3,0,01.01.17,Org1,Energy source 1 - Variable 3,0.1
4,Obj1,Energy source 1,Variable_4,0,01.01.17,Org1,Energy source 1 - Variable 4,0.1
5,Obj1,Energy source 1,Variable_5,0,01.01.17,Org1,Energy source 1 - Variable 5,0.1
6,Obj1,Energy source 1,Variable_6,0,01.01.17,Org1,Energy source 1 - Variable 6,0.1
7,Obj1,Energy source 2,Consumption,0,01.01.17,Org1,Energy source 2 - Consumption,0.1
8,Obj1,Energy source 2,Variable_1,0,01.01.17,Org1,Energy source 2 - Variable 1,0.1
9,Obj1,Energy source 2,Variable_2,0,01.01.17,Org1,Energy source 2 - Variable 2,0.1


In [4]:
df.dtypes

Object                object
Energy_Source         object
Feature               object
Value                  int64
Date                  object
Organization          object
Cube_Variable         object
p_value_threshold    float64
dtype: object

In [5]:
df.Date=pd.to_datetime(df.Date, format='%d.%m.%y') # преобразуем Date из строки в дату
df.sort_values(["Object", "Energy_Source", "Feature", "Date"], inplace=True)

threshold = df['p_value_threshold'].unique()[0]	# граница значимости коэффициента лин регрессии
organizations = df["Organization"].unique().tolist() # список организаций
output_DF_cols = ["Object", "Cube_Variable", "r2_adj", "Intercept", "Significance", "coef", "p_value", "Usefull"]

FULL_DFS = [] # объект хранения результатов расчетов всех моделей

In [6]:
for organization in organizations:
    print(organization)

    out_dfs = [] # список с результатами расчетов по срезам объект и энергоресурс
    
    organization_DF = df[(df["Organization"] == organization)] # берем срез по организации
    
    objects = list(set(organization_DF["Object"].tolist())) # выделяем объекты
    sources  = list(set(organization_DF["Energy_Source"].tolist())) # выделяем энергоресурсы
    combination = list(itertools.product(objects, sources)) # находим уникальные сочетания

    for object, source in combination:
        print((object, source))
        # берем срез
        initialDF = organization_DF[(organization_DF["Object"] == object) & (organization_DF["Energy_Source"] == source)] # срезаем данные по объекту и энергоресурсу

        Cube_Variables =initialDF['Cube_Variable'].unique().tolist()

        new_DF = make_new_DF(initialDF) # формируем датафрейм с необходимой структурой
        new_DF = fill_new_DF(initialDF, new_DF, object, source) # заполняем его данными
        new_DF = new_DF[(new_DF["Consumption"] > 0)] # отбрасываем значения с нулевым потреблением

        # готовим обучающие данные для модели
        y = new_DF.Consumption
        X = new_DF[new_DF.columns[3:]]
        X = sm.add_constant(X)

        # строим и обучаем линейную регрессию
        try:
            model = sm.OLS(y, X).fit()
            print("OK\n")
            output_DF = generate_out_df(model, Cube_Variables, object, output_DF_cols) # выводим результаты модели в отдельный датафрейм
        except ValueError:
            print("Blank DF\n")
            output_DF = generate_blank_out_df(Cube_Variables, object, output_DF_cols)

        out_dfs.append(output_DF)  # сохраняем результаты расчет по объекту и энергоресурсу
    
    full_output_DF = pd.concat(out_dfs, ignore_index=True) # объединение результатов расчета по объектам и энергоресурсам в один датафрейм
    full_output_DF['Organization'] = organization

    FULL_DFS.append(full_output_DF) # сохраняем расчеты по организации
    
FINAL_DF = pd.concat(FULL_DFS, ignore_index=True) # объединяем расчеты по организациям в один объект

Org1
('Obj1', 'Energy source 1')
OK

('Obj1', 'Energy source 3')
Blank DF

('Obj1', 'Energy source 8')
Blank DF

('Obj1', 'Energy source 10')
Blank DF

('Obj1', 'Energy source 7')
Blank DF

('Obj1', 'Energy source 5')
Blank DF

('Obj1', 'Energy source 6')
Blank DF

('Obj1', 'Energy source 9')
Blank DF

('Obj1', 'Energy source 4')
Blank DF

('Obj1', 'Energy source 2')
Blank DF

('Obj3', 'Energy source 1')
Blank DF

('Obj3', 'Energy source 3')
Blank DF

('Obj3', 'Energy source 8')
Blank DF

('Obj3', 'Energy source 10')
Blank DF

('Obj3', 'Energy source 7')
Blank DF

('Obj3', 'Energy source 5')
Blank DF

('Obj3', 'Energy source 6')
Blank DF

('Obj3', 'Energy source 9')
Blank DF

('Obj3', 'Energy source 4')
Blank DF

('Obj3', 'Energy source 2')
Blank DF

('Obj2', 'Energy source 1')
Blank DF

('Obj2', 'Energy source 3')
Blank DF

('Obj2', 'Energy source 8')
Blank DF

('Obj2', 'Energy source 10')
Blank DF

('Obj2', 'Energy source 7')
Blank DF

('Obj2', 'Energy source 5')
Blank DF

('Obj2', '

Как можно заметить, в данных много нулевых значений и расчеты произвелись только для Obj1. Посмотрим...

In [7]:
FINAL_DF[FINAL_DF.Object=='Obj1']

,Object,Cube_Variable,r2_adj,Intercept,Significance,coef,p_value,Usefull,Organization
0,Obj1,Energy source 1 - Variable 1,NaN,NaN,NaN,86.440032,0.009860,1.0,Org1
1,Obj1,Energy source 1 - Variable 2,NaN,NaN,NaN,524.053332,0.038987,1.0,Org1
2,Obj1,Energy source 1 - Variable 3,NaN,NaN,NaN,0.000000,1.000000,0.0,Org1
3,Obj1,Energy source 1 - Variable 4,NaN,NaN,NaN,0.000000,1.000000,0.0,Org1
4,Obj1,Energy source 1 - Variable 5,NaN,NaN,NaN,0.000000,1.000000,0.0,Org1
5,Obj1,Energy source 1 - Variable 6,NaN,NaN,NaN,0.000000,1.000000,0.0,Org1
6,Obj1,Energy source 1 - Consumption,0.677439,378806.950115,0.002492,NaN,NaN,NaN,Org1
7,Obj1,Energy source 3 - Consumption,NaN,NaN,NaN,0.000000,NaN,NaN,Org1
8,Obj1,Energy source 3 - Variable 1,NaN,NaN,NaN,0.000000,NaN,NaN,Org1
9,Obj1,Energy source 3 - Variable 2,NaN,NaN,NaN,0.000000,NaN,NaN,Org1
